## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-28-20-50-23 +0000,nyt,Dozens of States Sue Trump Administration Over...,https://www.nytimes.com/2025/10/28/us/politics...
1,2025-10-28-20-50-00 +0000,wsj,Nvidia CEO Praises Trump in MAGA-Themed Speech...,https://www.wsj.com/tech/ai/nvidia-ceo-praises...
2,2025-10-28-20-48-00 +0000,wsj,PPG Cuts Full-Year Earnings Outlook on Soft De...,https://www.wsj.com/business/earnings/ppg-cuts...
3,2025-10-28-20-47-10 +0000,nypost,"UPS axes 48,000 workers in sweeping cost-cut p...",https://nypost.com/2025/10/28/business/ups-axe...
4,2025-10-28-20-46-22 +0000,nyt,Live Updates: Hurricane Melissa Roars Across J...,https://www.nytimes.com/live/2025/10/28/weathe...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2333/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,47
90,new,25
38,hurricane,20
39,melissa,19
42,jamaica,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
190,2025-10-28-12-01-13 +0000,wapo,Japan’s new leader tries to charm Trump with g...,https://www.washingtonpost.com/politics/2025/1...,111
251,2025-10-28-07-06-06 +0000,bbc,China controls the rare earths the world buys ...,https://www.bbc.com/news/articles/cj6ny24j0r3o...,102
192,2025-10-28-11-49-55 +0000,nypost,Toyota pledges $10B toward new auto plants in ...,https://nypost.com/2025/10/28/us-news/trump-to...,101
116,2025-10-28-16-33-53 +0000,nypost,Mark Zuckerberg secretly met with AG Pam Bondi...,https://nypost.com/2025/10/28/business/mark-zu...,94
77,2025-10-28-18-16-09 +0000,nyt,Trump Administration Backs Plan for New Nuclea...,https://www.nytimes.com/2025/10/28/business/en...,92


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
190,111,2025-10-28-12-01-13 +0000,wapo,Japan’s new leader tries to charm Trump with g...,https://www.washingtonpost.com/politics/2025/1...
316,85,2025-10-27-21-54-59 +0000,nypost,Storm chaser captures stunning eye of Hurrican...,https://nypost.com/2025/10/27/world-news/hurri...
127,57,2025-10-28-16-09-00 +0000,wsj,The Republican-led House Oversight Committee i...,https://www.wsj.com/politics/policy/republican...
74,56,2025-10-28-18-19-57 +0000,nypost,Experts fear betting markets could influence N...,https://nypost.com/2025/10/28/business/pro-mam...
85,45,2025-10-28-17-53-26 +0000,wapo,U.S. military strikes four alleged drug boats ...,https://www.washingtonpost.com/national-securi...
295,42,2025-10-27-23-52-00 +0000,wsj,"“Both political parties have made their point,...",https://www.wsj.com/politics/policy/union-pres...
23,42,2025-10-28-20-20-26 +0000,nypost,Fuming SNAP recipients threaten to loot if foo...,https://nypost.com/2025/10/28/us-news/tiktokke...
135,41,2025-10-28-15-26-21 +0000,nypost,Bill Gates makes major climate change reversal...,https://nypost.com/2025/10/28/us-news/bill-gat...
37,40,2025-10-28-19-49-44 +0000,nypost,Disturbing video shows Hamas stage recovery of...,https://nypost.com/2025/10/28/world-news/hamas...
7,38,2025-10-28-20-39-00 +0000,wsj,"Cheesecake Factory Third-Quarter Profit Up, Co...",https://www.wsj.com/business/earnings/cheeseca...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
